In [ ]:
import pandas as pd

In [ ]:
df_test = pd.read_csv('test_raw.txt')
df_test = df_test.rename(columns={'Location.Address.PostalCode': 'Zip'})
df_test['Zip'] = df_test['Zip'].astype(str).str[:5].astype(int)

In [ ]:
df_income = pd.read_excel('income.xlsx')
df_income['Zip / Population'] = df_income['Zip / Population'].str[:5]
df_income= df_income.drop(columns=['Rank'])
df_income = df_income.rename(columns={'Zip / Population': 'Zip'})
df_income = df_income.rename(columns={'Median Household Income ▼': 'Median Household Income'})
df_income['Zip'] = df_income['Zip'].astype(int)
df_income['Median Household Income'] = (
    df_income['Median Household Income']
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(int)  )
df_income.head()

In [ ]:
df = pd.merge(df_test, df_income, on='Zip', how='left')
df.to_csv('data_with_income.csv', index=False)

In [ ]:
df['ImageData.c1c6.summary.property'] = df['ImageData.c1c6.summary.property'].fillna(df['ImageData.c1c6.summary.property'].mean())
df['ImageData.q1q6.summary.property'] = df['ImageData.q1q6.summary.property'].fillna(df['ImageData.q1q6.summary.property'].mean())
df['Structure.Rooms.RoomsTotal'] = df['Structure.Rooms.RoomsTotal'].fillna(df['Structure.Rooms.RoomsTotal'].mean())

In [ ]:
import ast

# List of the categorical columns that are structure as a list of strings
columns_to_transform = ['ImageData.features_reso.results', 'ImageData.room_type_reso.results',
                        'Structure.Heating', 'Structure.Cooling']

# Separate each list
for col in columns_to_transform:
    df[col] = [
        ast.literal_eval(sublist) if isinstance(sublist, str) else []
        for sublist in df[col]
    ]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

mlb = MultiLabelBinarizer()


# Apply one-hot encoding to each column and concatenate the results
one_hot_features = pd.DataFrame(mlb.fit_transform(df['ImageData.features_reso.results']),
                                 columns=mlb.classes_, index=df.index)

one_hot_room_type = pd.DataFrame(mlb.fit_transform(df['ImageData.room_type_reso.results']),
                                 columns=mlb.classes_, index=df.index)

one_hot_heating = pd.DataFrame(mlb.fit_transform(df['Structure.Heating']),
                                 columns=mlb.classes_, index=df.index)

one_hot_cooling = pd.DataFrame(mlb.fit_transform(df['Structure.Cooling']),
                                 columns=mlb.classes_, index=df.index)

# Concatenate the result in the the original dataset
one_hot_categorical = pd.concat([one_hot_features, one_hot_room_type, one_hot_heating,one_hot_cooling], axis=1)

In [ ]:
one_hot_categorical.columns

In [ ]:
columns_to_keep = ['Appliances.BuiltInRefrigerator',
       'Appliances.Cooktop', 'Appliances.DoubleOven', 'Appliances.GasCooktop',
       'Appliances.Oven', 'Appliances.WineCooler', 'Heating.Fireplaces',
       'InteriorOrRoomFeatures.BreakfastBar',
       'InteriorOrRoomFeatures.BuiltInFeatures',
       'InteriorOrRoomFeatures.DoubleVanity',
       'InteriorOrRoomFeatures.KitchenIsland',
       'InteriorOrRoomFeatures.TrayCeilings']

one_hot_reso = one_hot_categorical[columns_to_keep]
df_reso = pd.concat([df, one_hot_reso], axis=1)


In [ ]:
encoder = OneHotEncoder()

# Apply one-hot encoding 
one_hot = encoder.fit_transform(df[['Property.PropertyType']])
one_hot = one_hot.toarray()

encoded_property_type_df = pd.DataFrame(one_hot, columns=encoder.get_feature_names_out(['Property.PropertyType']))

df_expanded_prev = pd.concat([df_reso, encoded_property_type_df], axis=1)
df_expanded = df_expanded_prev.drop(columns=['Property.PropertyType', 'ImageData.features_reso.results', 'ImageData.room_type_reso.results','Structure.GarageSpaces'])

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np

train_data = pd.read_csv('cleaned_data_without_scaling.csv')

columns_for_knn = list(encoded_property_type_df.columns) + ['Zip',
                  'Structure.BathroomsFull', 'Structure.BathroomsHalf', 'Structure.BedroomsTotal',
                  'Structure.Rooms.RoomsTotal', 'Median Household Income']

knn_data = train_data[columns_for_knn]

knn_imputer = KNNImputer(n_neighbors=5)

# Train the imputer with train_data
knn_imputer.fit(knn_data)

# Apply the trained imputer to df_expanded
imputed_data = knn_imputer.transform(df_expanded[columns_for_knn])

# Create a DataFrame with the imputed data
imputed_df = pd.DataFrame(imputed_data, columns=columns_for_knn, index=df_expanded.index)

# Update df_expanded with the imputed values
df_expanded[columns_for_knn] = imputed_df[columns_for_knn]


In [ ]:
columns_for_knn = list(encoded_property_type_df.columns) + ['Median Household Income',
                  'Structure.BathroomsFull', 'Structure.BathroomsHalf', 'Structure.BedroomsTotal',
                  'Characteristics.SurfaceArea']

knn_data = train_data[columns_for_knn]

imputer2 = KNNImputer(n_neighbors=5)

imputer2.fit(knn_data)

# Apply the trained imputer to df_expanded
imputed_data = imputer2.transform(df_expanded[columns_for_knn])

# Create a DataFrame with the imputed data
imputed_df = pd.DataFrame(imputed_data, columns=columns_for_knn, index=df_expanded.index)

# Update df_expanded with the imputed values for 'Characteristics.SurfaceArea'
df_expanded['Characteristics.SurfaceArea'] = imputed_df['Characteristics.SurfaceArea']

In [ ]:
df_expanded= df_expanded.drop(columns=['Listing.Dates.CloseDate','Zip','Structure.Cooling', 'Structure.Heating'])


In [ ]:
nan_count = df_expanded.isna().sum()
nan_count
df_expanded.shape

In [ ]:
df_expanded.to_csv('nanfree_test.csv', index=False)